In [3]:
import dask.distributed
import dask.utils
import numpy as np
import planetary_computer as pc
import xarray as xr
from IPython.display import display
from pystac_client import Client

from shapely.geometry import Polygon
from odc.stac import configure_rio, stac_load


class Sentinel2DataFetcher:
    def __init__(self):

        client = dask.distributed.Client()
        configure_rio(cloud_defaults=True, client=client)
        display(client)

        self.plancomp_catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")


    def search_sentinel2(self, start_date, end_date, aoi):
        """
        Search for Sentinel-2 data within a time period and area of interest.

        Parameters:
        - start_date (str): Start date in the format "YYYY-MM-DD".
        - end_date (str): End date in the format "YYYY-MM-DD".
        - aoi (list): Area of interest as a list of (longitude, latitude) tuples.

        Returns:
        - A list of items (dicts) matching the search criteria.
        """
        # Create a polygon from the area of interest
        geometry = Polygon(aoi)

        # Search for data in the Sentinel-2 collection that intersects with the geometry
        # during the specified time period
        search = self.catalog.search(
            collections=['sentinel-2-l2a'],
            datetime=f"{start_date}/{end_date}",
            intersects=geometry,
            # limit=10  # Adjust limit as necessary
        )

        # Fetch the items
        items = list(search.get_items())
        print(f"Found {len(items)} items.")

        # Extract the STAC items and return them
        return [item.to_dict() for item in items]

In [ ]:
# Example usage:
fetcher = Sentinel2DataFetcher()

# Define the area of interest as a list of (lon, lat) tuples, forming a closed loop.
area_of_interest = [
    (-105.0, 40.0),
    (-105.0, 45.0),
    (-100.0, 45.0),
    (-100.0, 40.0),
    (-105.0, 40.0)  # Closing the loop
]

# Define the time period
start_date = '2022-01-01'
end_date = '2022-01-31'

# Perform the search
sentinel2_data = fetcher.search_sentinel2(start_date, end_date, area_of_interest)

# The 'sentinel2_data' variable now holds the search results.

In [6]:
import boto3
from datetime import datetime, timedelta

class Sentinel2DataFetcherAWS:
    def __init__(self):
        self.s3_client = boto3.client('s3')
        self.bucket_name = 'sentinel-s2-l1c'

    def search_sentinel2(self, start_date, end_date, tile_prefix):
        """
        List available Sentinel-2 scenes from the AWS S3 bucket for a given time period and tile prefix.

        Parameters:
        - start_date (str): Start date in the format "YYYY-MM-DD".
        - end_date (str): End date in the format "YYYY-MM-DD".
        - tile_prefix (str): Tile prefix (e.g., "tiles/10/S/DG/") for the area of interest.

        Returns:
        - A list of available scene keys in the S3 bucket.
        """
        # Format dates
        start_datetime = datetime.strptime(start_date, "%Y-%m-%d")
        end_datetime = datetime.strptime(end_date, "%Y-%m-%d")
        
        # Generate a list of dates (as strings) between start_date and end_date
        date_list = [start_datetime + timedelta(days=x) for x in range(0, (end_datetime-start_datetime).days)]
        date_strings = [date.strftime("%Y/%m/%d") for date in date_list]

        # Initialize an empty list to store the keys of available scenes
        scene_keys = []

        # Iterate over each date and list the objects in the S3 bucket
        for date_string in date_strings:
            prefix = f"{tile_prefix}{date_string}/"
            response = self.s3_client.list_objects_v2(Bucket=self.bucket_name, Prefix=prefix)

            # If objects are found for the date, add their keys to the scene_keys list
            if 'Contents' in response:
                for obj in response['Contents']:
                    scene_keys.append(obj['Key'])

        return scene_keys

In [7]:
# Example usage
fetcher = Sentinel2DataFetcherAWS()

# Define the time period
start_date = '2022-01-01'
end_date = '2022-01-31'

# Define the tile prefix for your area of interest
# Example tile prefix: 'tiles/10/S/DG/'
tile_prefix = 'tiles/10/S/DG/'

# Perform the search
sentinel2_scenes = fetcher.search_sentinel2(start_date, end_date, tile_prefix)

# The 'sentinel2_scenes' variable now holds the keys for the available scenes.

NoCredentialsError: Unable to locate credentials